In [6]:
from serde import serde
from serde.json import to_json, from_json
from dataclasses import dataclass
from time import sleep
from veron_nepveux_project.scrapping import Voiture

ModuleNotFoundError: No module named 'veron_nepveux_project'

In [2]:
with open("donnees.json", "r") as fichier:
    contenu_fichier = fichier.read()

In [3]:
print(contenu_fichier)

{"car1":{"modele":"3008","carburant":"Essence","prix":32470,"kilometrage":7304,"garantie_kilometrage":"non garanti","boite_de_vitesse":"Automatique","transmission":"2","couleur":"Gris","garantie":"SPOTICAR PREMIUM","date_mise_circulation":2022,"puissance":131,"silhouette":"SUV-4x4","nb_places":5,"utilisation_prec":"NA","puissance_fiscale":7,"critair":"1","ptac":1940,"nb_portes":5},"car2":{"modele":"308","carburant":"Diesel","prix":14970,"kilometrage":71395,"garantie_kilometrage":"garanti","boite_de_vitesse":"Manuelle","transmission":"2","couleur":"Gris","garantie":"SPOTICAR PREMIUM","date_mise_circulation":2017,"puissance":120,"silhouette":"Berline","nb_places":5,"utilisation_prec":"E","puissance_fiscale":5,"critair":"2","ptac":1810,"nb_portes":5},"car3":{"modele":"3008","carburant":"Diesel","prix":36470,"kilometrage":3554,"garantie_kilometrage":"non garanti","boite_de_vitesse":"Automatique","transmission":"2","couleur":"Gris","garantie":"SPOTICAR PREMIUM","date_mise_circulation":2022,